In [ ]:
#  data exploration run inference on medical model
import torch
from transformers import  AutoModelForCausalLM
from transformers import AutoTokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# medi_tokenizer = AutoTokenizer.from_pretrained("epfl-llm/meditron-70b")
# meditron_model = AutoModelForCausalLM.from_pretrained("epfl-llm/meditron-70b")


In [ ]:
from huggingface_hub import login
token = 'hugging face key'
login(token=token)



In [ ]:
import pandas as pd

splits = {'train': 'phrases_no_exclude_train.jsonl', 'test': 'phrases_no_exclude_test.jsonl'}
medqa4_df = pd.read_json("hf://datasets/GBaker/MedQA-USMLE-4-options/" + splits["test"], lines=True)


print(len(medqa4_df))

1273


In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'data/train-00000-of-00001.parquet', 'valid': 'data/valid-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
pubmed_df = pd.read_parquet("hf://datasets/clinicalnlplab/pubmedqa_test/" + splits["test"])

print(len(pubmed_df))
print(pubmed_df.head())
print(pubmed_df['query'][0])


500
          id                                              query answer  \
0  PubmedQA0  As an expert doctor in clinical science and me...    yes   
1  PubmedQA1  As an expert doctor in clinical science and me...    yes   
2  PubmedQA2  As an expert doctor in clinical science and me...    yes   
3  PubmedQA3  As an expert doctor in clinical science and me...    yes   
4  PubmedQA4  As an expert doctor in clinical science and me...    yes   

            choices  gold  
0  [yes, no, maybe]     0  
1  [yes, no, maybe]     0  
2  [yes, no, maybe]     0  
3  [yes, no, maybe]     0  
4  [yes, no, maybe]     0  
As an expert doctor in clinical science and medical knowledge, can you tell me if the following statement is correct? Answer yes, no, or maybe.
Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the musculus (m.) puborectalis in patients with dyschesia using anorectal endoso

In [ ]:
print(medqa4_df.head())

                                            question  \
0  A junior orthopaedic surgery resident is compl...   
1  A 67-year-old man with transitional cell carci...   
2  Two weeks after undergoing an emergency cardia...   
3  A 39-year-old woman is brought to the emergenc...   
4  A 35-year-old man comes to the physician becau...   

                                              answer  \
0  Tell the attending that he cannot fail to disc...   
1                               Cross-linking of DNA   
2                           Cholesterol embolization   
3  Lactose-fermenting, gram-negative rods forming...   
4                                Ketotifen eye drops   

                                             options meta_info answer_idx  \
0  {'A': 'Disclose the error to the patient and p...     step1          B   
1  {'A': 'Inhibition of proteasome', 'B': 'Hypers...     step1          D   
2  {'A': 'Renal papillary necrosis', 'B': 'Choles...   step2&3          B   
3  {'A': 'Coagulas

In [ ]:
from huggingface_hub import InferenceClient

# Initialize client with your Hugging Face token
client = InferenceClient(token=token, provider="groq",)

# Example list of 50 prompts (replace with your actual queries)
prompts = []

for q, opts in zip(medqa4_df['question'], medqa4_df['options']):
    # Convert options list to string
    options_str = ", ".join(opts)

    # Create a refined prompt
    prompt = f"""
    Answer the following question by choosing ONLY from A, B, C, D, or E.
    Do NOT add any explanation or extra words.

    Question: {q}

    Choices:
    A. {opts['A']}
    B. {opts['B']}
    C. {opts['C']}
    D. {opts['D']}

    Answer:
    """
    prompts.append(prompt)

In [ ]:
from huggingface_hub import InferenceClient

# Initialize client with your Hugging Face token
client = InferenceClient(token=token, provider="groq",)

# Example list of 50 prompts (replace with your actual queries)
prompts = []

for q in pubmed_df['query']:
    # Convert options list to string
    options_str = ", ".join(opts)

    # Create a refined prompt
    prompt = q
    prompts.append(prompt)

In [ ]:

from tqdm import tqdm
# Function to send batch requests
def batch_evaluate(prompts, max_new_tokens=200, temperature=0.0):
    results = []
    for prompt in tqdm(prompts, "processing prompts "):
        try:

            response = client.chat_completion(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[{"role": "user", "content": prompt}],

              )
              # Extract the generated text
            # # text = response.choices[0].message.content
            # print(text)
            results.append(response)
        except Exception as e:
            # Handle errors gracefully
            print(e)
    return results

# Evaluate all queries
outputs = batch_evaluate(prompts)

# Print results
for i, out in enumerate(outputs):
    print(f"Prompt {i+1}: {out}")


processing prompts : 100%|██████████| 1273/1273 [05:35<00:00,  3.79it/s]


Prompt 1: ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='A.', reasoning=None, tool_call_id=None, tool_calls=None), logprobs=None)], created=1763563637, id='chatcmpl-f38f972c-cdd5-4398-8486-29e6d8be2487', model='meta-llama/llama-4-scout-17b-16e-instruct', system_fingerprint='fp_f08cd12a81', usage=ChatCompletionOutputUsage(completion_tokens=3, prompt_tokens=219, total_tokens=222, queue_time=0.220616862, prompt_time=0.011168887, completion_time=0.007127529, total_time=0.018296416), object='chat.completion', usage_breakdown=None, x_groq={'id': 'req_01kae9b24wfcrvh5x6a0tdggt9', 'seed': 1444504022}, service_tier='on_demand')
Prompt 2: ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='D.', reasoning=None, tool_call_id=None, tool_calls=None), logprobs=None)], created=1763563637, id='chatc

In [ ]:

from tqdm import tqdm
# Function to send batch requests
def batch_evaluate(prompts, max_new_tokens=200, temperature=0.0):
    results = []
    for prompt in tqdm(prompts, "processing prompts "):
        try:

            response = client.chat_completion(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[{"role": "user", "content": prompt}],

              )
              # Extract the generated text
            # # text = response.choices[0].message.content
            # print(text)
            results.append(response)
        except Exception as e:
            # Handle errors gracefully
            print(e)
    return results

# Evaluate all queries
outputs = batch_evaluate(prompts)

# Print results
for i, out in enumerate(outputs):
    print(f"Prompt {i+1}: {out}")


processing prompts : 100%|██████████| 500/500 [04:33<00:00,  1.83it/s]

Prompt 1: ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Yes. \n\nThe study demonstrates that anorectal endosonography can detect dysfunction of the anal sphincter and/or the musculus puborectalis in patients with dyschesia, which suggests that this diagnostic tool is valuable in assessing and potentially managing this condition.', reasoning=None, tool_call_id=None, tool_calls=None), logprobs=None)], created=1763571286, id='chatcmpl-dae76b5f-ebfa-4612-95bf-0410ab7361ab', model='meta-llama/llama-4-scout-17b-16e-instruct', system_fingerprint='fp_5436ed2ebe', usage=ChatCompletionOutputUsage(completion_tokens=55, prompt_tokens=340, total_tokens=395, queue_time=0.081524358, prompt_time=0.011541154, completion_time=0.124791987, total_time=0.136333141), object='chat.completion', usage_breakdown=None, x_groq={'id': 'req_01kaegmgcmfxwb24m8jp5gaxcg', 'seed': 1648483936}, service_tier='on_dem

In [ ]:
for i, out in enumerate(outputs):
    print(f"Prompt {i+1}: {out.choices[0].message.content}")

Prompt 1: Yes. 

The study demonstrates that anorectal endosonography can detect dysfunction of the anal sphincter and/or the musculus puborectalis in patients with dyschesia, which suggests that this diagnostic tool is valuable in assessing and potentially managing this condition.
Prompt 2: Yes.
Prompt 3: Based on the information provided, the answer is: Yes.

The study found a close correlation (r = 0.84) between tacrolimus concentrations measured by the affinity column-mediated immunoassay (ACMIA) method and the microparticle enzyme immunoassay (MEIA) method. Additionally, the Bland-Altman plot showed no specific trends, suggesting that the two methods are comparable. While there was a tendency for the difference between the two methods to be larger in low hematocrit samples, the results suggest that the ACMIA method can be used as an alternative to the MEIA method for measuring blood tacrolimus levels.
Prompt 4: Yes. 

The statement in the text indicates that physicians' specialtie

In [ ]:
answers=[]
import re
for i, out in enumerate(outputs):
    match = re.search(r'\b(A|B|C|D)\b', out.choices[0].message.content)
    answer = match.group(0) if match else None
    answers.append(answer)
print(answers)
print(sum(x is not None for x in answers))

['A', 'D', 'B', 'B', 'B', 'B', 'A', 'C', 'B', 'A', 'D', 'D', 'B', 'D', 'C', 'D', 'D', 'D', 'B', 'D', 'C', 'A', 'C', 'A', 'A', 'D', 'D', 'A', 'D', 'A', 'D', 'D', 'C', 'D', 'D', 'A', 'D', 'A', 'D', 'A', 'C', 'C', 'A', 'C', 'C', 'D', 'B', 'A', 'B', 'C', 'D', 'D', 'D', 'C', 'A', 'A', 'C', 'B', 'C', 'C', 'C', 'A', 'A', 'A', 'C', 'B', 'C', 'B', 'B', 'C', 'A', 'A', 'C', 'B', 'A', 'A', 'B', 'B', 'C', 'C', 'B', 'D', 'C', 'A', 'C', 'B', 'C', 'C', 'C', 'B', 'D', 'B', 'C', 'C', 'C', 'C', 'C', 'B', 'B', 'B', 'D', 'A', 'A', 'A', 'B', 'B', 'A', 'A', 'D', 'C', 'C', 'B', 'A', 'C', 'B', 'D', 'C', 'C', 'D', 'C', 'D', 'A', 'D', 'A', 'B', 'A', 'A', 'C', 'B', 'A', 'B', 'B', 'A', 'B', 'A', 'C', 'C', 'D', 'B', 'B', 'C', 'A', 'A', 'B', 'C', 'B', 'C', 'D', 'B', 'D', 'B', 'D', 'A', 'A', 'C', 'D', 'B', 'A', 'C', 'D', 'C', 'B', 'C', 'A', 'A', 'B', 'B', 'C', 'B', 'C', 'C', 'D', 'B', 'A', 'D', 'D', 'D', 'C', 'C', 'C', 'C', 'D', 'B', 'D', 'A', 'D', 'B', 'B', 'D', 'C', 'A', 'A', 'D', 'C', 'C', 'C', 'B', 'B', 'C', 'C',

In [ ]:
answers=[]
import re
for i, out in enumerate(outputs):
    match = re.search(r'\b(A|B|C|D)\b', out.choices[0].message.content)
    answer = match.group(0) if match else None
    answers.append(answer)
print(answers)
print(sum(x is not None for x in answers))

[None, None, None, None, None, None, None, 'A', None, None, None, 'A', None, None, None, None, None, None, None, None, None, None, 'A', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'D', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'A', None, None, None, None, None, None, None, None, 'A', 'A', None, None, None, None, None, None, 'C', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'A', None, None, None, None, None, 'A', None, None, None, None, None, None, None, None, None, None, 'A', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, No

In [ ]:

from sklearn.metrics import accuracy_score

medqa4_df['answer_idx'] = medqa4_df['answer_idx'].astype(str).str.upper()

print(medqa4_df['answer_idx'])

y_pred = [p if p is not None else "N" for p in answers]


y_true = medqa4_df['answer_idx'].astype(str).tolist()

acc = accuracy_score(y_pred,y_true)
print(f"medqa llama4:{acc}")

0       B
1       D
2       B
3       D
4       B
       ..
1268    D
1269    D
1270    B
1271    C
1272    C
Name: answer_idx, Length: 1273, dtype: object
medqa llama4:0.7494108405341713


In [ ]:
from sklearn.metrics import accuracy_score

# pubmed_df['gold'] = pubmed_df['answer_idx'].astype(str).str.upper()

# print(pubmed_df['answer_idx'])

y_pred = [p if p is not None else "N" for p in answers]


y_true = pubmed_df['gold'].astype(str).tolist()

acc = accuracy_score(y_pred,y_true)
print(f"medqa llama4:{acc}")

medqa llama4:0.0
